# Book Recommendation System

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

## Load the Datasets

The dataset is divided into three files:
*   `BX-Books.csv`: Information about the books (ISBN, title, author, etc.).
*   `BX-Book-Ratings.csv`: User ratings for the books.
*   `BX-Users.csv`: Information about the users.

In [2]:
data_dir = Path("../data/ingested_data")

In [24]:
# Load the dataset
books = pd.read_csv(data_dir / "BX-Books.csv", sep=';', encoding='latin-1', on_bad_lines='skip', low_memory=False)
ratings = pd.read_csv(data_dir / "BX-Book-Ratings.csv", sep=';', encoding='latin-1', on_bad_lines='skip')
users = pd.read_csv(data_dir / "BX-Users.csv", sep=';', encoding='latin-1', on_bad_lines='skip')

### Books Dataset

In [4]:
books.head(2)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...


### Ratings Dataset

In [5]:
ratings.head(2)

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5


### Users Dataset

In [6]:
users.head(2)

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0


## Data Exploration

### Shape of the Datasets

In [7]:
print(f"Books dataset shape: {books.shape}")
print(f"Ratings dataset shape: {ratings.shape}")
print(f"Users dataset shape: {users.shape}")

Books dataset shape: (271360, 8)
Ratings dataset shape: (1149780, 3)
Users dataset shape: (278858, 3)


### Missing Values

In [8]:
print("Missing values in Books dataset:")
print(books.isnull().sum())

Missing values in Books dataset:
ISBN                   0
Book-Title             0
Book-Author            2
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64


In [9]:
print("\nMissing values in Ratings dataset:")
print(ratings.isnull().sum())


Missing values in Ratings dataset:
User-ID        0
ISBN           0
Book-Rating    0
dtype: int64


In [10]:
print("\nMissing values in Users dataset:")
print(users.isnull().sum())


Missing values in Users dataset:
User-ID          0
Location         0
Age         110762
dtype: int64


The `users` dataset has a large number of missing values in the `Age` column. We will need to handle this.

### Duplicate Values

In [12]:
print(f"Duplicate values in Books dataset: {books.duplicated().sum()}")
print(f"Duplicate values in Ratings dataset: {ratings.duplicated().sum()}")
print(f"Duplicate values in Users dataset: {users.duplicated().sum()}")

Duplicate values in Books dataset: 0
Duplicate values in Ratings dataset: 0
Duplicate values in Users dataset: 0


## Data Cleaning and Preprocessing

Let's clean up the column names to make them easier to work with.

In [ ]:
# Drop the small and medium size image columns and raname the features
books = books[['ISBN','Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher','Image-URL-L']]
books.rename(columns={"Book-Title":'title',
                      'Book-Author':'author',
                     "Year-Of-Publication":'year',
                     "Publisher":"publisher",
                     "Image-URL-L":"image_url"},inplace=True)


In [27]:
books.columns

Index(['ISBN', 'title', 'author', 'year', 'publisher', 'image_url'], dtype='object')

In [ ]:
# Raname the column
users.rename(columns={"User-ID":'user_id',
                      'Location':'location',
                     "Age":'age'},inplace=True)


In [29]:
users.columns

Index(['user_id', 'location', 'age'], dtype='object')

In [31]:
# Rename the column
ratings.rename(columns={"User-ID":'user_id',
                      'Book-Rating':'rating'},inplace=True)


In [32]:
ratings.columns

Index(['user_id', 'ISBN', 'rating'], dtype='object')

In [33]:
ratings['user_id'].value_counts()

user_id
11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
119573        1
276706        1
276697        1
276679        1
276676        1
Name: count, Length: 105283, dtype: int64

In [34]:
# store users who had at least rated more than 200 books
x = ratings['user_id'].value_counts() > 200

In [35]:
x

user_id
11676      True
198711     True
153662     True
98391      True
35859      True
          ...  
119573    False
276706    False
276697    False
276679    False
276676    False
Name: count, Length: 105283, dtype: bool

In [ ]:
x[x].shape

(899,)

In [ ]:
y= x[x].index

In [38]:
y

Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352, 110973,
       235105,
       ...
       116122,  44296,  28634,  59727,  73681, 274808, 188951,   9856, 155916,
       268622],
      dtype='int64', name='user_id', length=899)

In [39]:
ratings = ratings[ratings['user_id'].isin(y)]

In [40]:
ratings.head()

,user_id,ISBN,rating
1456,277427,002542730X,10
1457,277427,0026217457,0
1458,277427,003008685X,8
1459,277427,0030615321,0
1460,277427,0060002050,0


In [41]:
# Join ratings with books

ratings_with_books = ratings.merge(books, on='ISBN')

In [43]:
ratings_with_books.head(2)

,user_id,ISBN,rating,title,author,year,publisher,image_url
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
1,277427,0026217457,0,Vegetarian Times Complete Cookbook,Lucy Moll,1995,John Wiley &amp; Sons,http://images.amazon.com/images/P/0026217457.0...


In [44]:
number_rating = ratings_with_books.groupby('title')['rating'].count().reset_index()

In [45]:
number_rating.head()

,title,rating
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1


In [46]:
number_rating.rename(columns={'rating':'num_of_rating'},inplace=True)

In [47]:
number_rating.head()

,title,num_of_rating
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1


In [48]:
final_rating = ratings_with_books.merge(number_rating, on='title')

In [49]:
final_rating.head(2)

,user_id,ISBN,rating,title,author,year,publisher,image_url,num_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
1,277427,0026217457,0,Vegetarian Times Complete Cookbook,Lucy Moll,1995,John Wiley &amp; Sons,http://images.amazon.com/images/P/0026217457.0...,7


In [50]:
# Filer those books which got at least 50 rating of user

final_rating = final_rating[final_rating['num_of_rating'] >= 50]


In [51]:
final_rating.head()

,user_id,ISBN,rating,title,author,year,publisher,image_url,num_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
13,277427,0060930535,0,The Poisonwood Bible: A Novel,Barbara Kingsolver,1999,Perennial,http://images.amazon.com/images/P/0060930535.0...,133
15,277427,0060934417,0,Bel Canto: A Novel,Ann Patchett,2002,Perennial,http://images.amazon.com/images/P/0060934417.0...,108
18,277427,0061009059,9,One for the Money (Stephanie Plum Novels (Pape...,Janet Evanovich,1995,HarperTorch,http://images.amazon.com/images/P/0061009059.0...,108
24,277427,006440188X,0,The Secret Garden,Frances Hodgson Burnett,1998,HarperTrophy,http://images.amazon.com/images/P/006440188X.0...,79


In [52]:
final_rating.isnull().sum()

user_id          0
ISBN             0
rating           0
title            0
author           0
year             0
publisher        0
image_url        0
num_of_rating    0
dtype: int64

In [53]:
# create a pivot table
book_pivot = final_rating.pivot_table(columns='user_id', index='title', values= 'rating')

In [54]:
book_pivot

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1st to Die: A Novel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2nd Chance,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN
4 Blondes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84 Charing Cross Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,7.0,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
You Belong To Me,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [55]:
book_pivot.shape

(742, 888)

In [56]:
book_pivot.fillna(0, inplace=True)

In [57]:
book_pivot.head(2)

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [58]:
from scipy.sparse import csr_matrix

In [59]:
book_sparse = csr_matrix(book_pivot)

In [60]:
book_sparse

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 15226 stored elements and shape (742, 888)>

In [61]:
type(book_sparse)

scipy.sparse._csr.csr_matrix

In [62]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm= 'brute')

In [63]:
model.fit(book_sparse)

,n_neighbors,5
,radius,1.0
,algorithm,'brute'
,leaf_size,30
,metric,'minkowski'
,p,2
,metric_params,None
,n_jobs,None


In [64]:
book_pivot.iloc[237,:]

user_id
254       9.0
2276      0.0
2766      0.0
2977      0.0
3363      0.0
         ... 
275970    9.0
277427    0.0
277478    0.0
277639    0.0
278418    0.0
Name: Harry Potter and the Chamber of Secrets (Book 2), Length: 888, dtype: float64

In [65]:
distance, suggestion = model.kneighbors(book_pivot.iloc[237,:].values.reshape(1,-1), n_neighbors=6 )

In [66]:
distance

array([[ 0.        , 67.73129098, 67.77802823, 72.22091879, 76.03909813,
        76.55027397]])

In [67]:
suggestion

array([[237, 240, 238, 241, 184, 291]])

In [68]:
for i in range(len(suggestion)):
    print(book_pivot.index[suggestion[i]])


Index(['Harry Potter and the Chamber of Secrets (Book 2)',
       'Harry Potter and the Prisoner of Azkaban (Book 3)',
       'Harry Potter and the Goblet of Fire (Book 4)',
       'Harry Potter and the Sorcerer's Stone (Book 1)', 'Exclusive',
       'Jacob Have I Loved'],
      dtype='object', name='title')


In [69]:
book_names = book_pivot.index

In [70]:
book_names

Index(['1984', '1st to Die: A Novel', '2nd Chance', '4 Blondes',
       '84 Charing Cross Road', 'A Bend in the Road', 'A Case of Need',
       'A Child Called \It\": One Child's Courage to Survive"',
       'A Civil Action', 'A Cry In The Night',
       ...
       'Winter Solstice', 'Wish You Well', 'Without Remorse',
       'Wizard and Glass (The Dark Tower, Book 4)', 'Wuthering Heights',
       'Year of Wonders', 'You Belong To Me',
       'Zen and the Art of Motorcycle Maintenance: An Inquiry into Values',
       'Zoya', '\O\" Is for Outlaw"'],
      dtype='object', name='title', length=742)

In [71]:
book_names[4]

'84 Charing Cross Road'

In [72]:
final_rating.head()

,user_id,ISBN,rating,title,author,year,publisher,image_url,num_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
13,277427,0060930535,0,The Poisonwood Bible: A Novel,Barbara Kingsolver,1999,Perennial,http://images.amazon.com/images/P/0060930535.0...,133
15,277427,0060934417,0,Bel Canto: A Novel,Ann Patchett,2002,Perennial,http://images.amazon.com/images/P/0060934417.0...,108
18,277427,0061009059,9,One for the Money (Stephanie Plum Novels (Pape...,Janet Evanovich,1995,HarperTorch,http://images.amazon.com/images/P/0061009059.0...,108
24,277427,006440188X,0,The Secret Garden,Frances Hodgson Burnett,1998,HarperTrophy,http://images.amazon.com/images/P/006440188X.0...,79


In [73]:
np.where(book_pivot.index == '4 Blondes')[0][0]

np.int64(3)

In [74]:
# final_rating['title'].value_counts()
ids = np.where(final_rating['title'] == "Harry Potter and the Chamber of Secrets (Book 2)")[0][0]

In [75]:
final_rating.iloc[ids]['image_url']

'http://images.amazon.com/images/P/0439064872.01.LZZZZZZZ.jpg'

In [76]:
book_name = []
for book_id in suggestion:
    book_name.append(book_pivot.index[book_id])


In [77]:
book_name

[Index(['Harry Potter and the Chamber of Secrets (Book 2)',
        'Harry Potter and the Prisoner of Azkaban (Book 3)',
        'Harry Potter and the Goblet of Fire (Book 4)',
        'Harry Potter and the Sorcerer's Stone (Book 1)', 'Exclusive',
        'Jacob Have I Loved'],
       dtype='object', name='title')]

In [78]:
ids_index = []
for name in book_name[0]: 
    ids = np.where(final_rating['title'] == name)[0][0]
    ids_index.append(ids)

In [79]:
ids_index

[np.int64(44),
 np.int64(45),
 np.int64(338),
 np.int64(46),
 np.int64(812),
 np.int64(895)]

In [80]:
for idx in ids_index:
    url = final_rating.iloc[idx]['image_url']
    print(url)

http://images.amazon.com/images/P/0439064872.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/0439136369.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/0439139597.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/043936213X.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/0446604232.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/0064403688.01.LZZZZZZZ.jpg


In [84]:
proj_dir = Path("../")

In [85]:
import pickle
pickle.dump(model,open(proj_dir/'artifacts/model.pkl','wb'))
pickle.dump(book_names,open(proj_dir/'artifacts/book_names.pkl','wb'))
pickle.dump(final_rating,open(proj_dir/'artifacts/final_rating.pkl','wb'))
pickle.dump(book_pivot,open(proj_dir/'artifacts/book_pivot.pkl','wb'))


In [86]:
book_names

Index(['1984', '1st to Die: A Novel', '2nd Chance', '4 Blondes',
       '84 Charing Cross Road', 'A Bend in the Road', 'A Case of Need',
       'A Child Called \It\": One Child's Courage to Survive"',
       'A Civil Action', 'A Cry In The Night',
       ...
       'Winter Solstice', 'Wish You Well', 'Without Remorse',
       'Wizard and Glass (The Dark Tower, Book 4)', 'Wuthering Heights',
       'Year of Wonders', 'You Belong To Me',
       'Zen and the Art of Motorcycle Maintenance: An Inquiry into Values',
       'Zoya', '\O\" Is for Outlaw"'],
      dtype='object', name='title', length=742)

### Testing the model

In [87]:
def recommend_book(book_name):
    book_id = np.where(book_pivot.index == book_name)[0][0]
    distance, suggestion = model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1,-1), n_neighbors=6 )
    
    for i in range(len(suggestion)):
            books = book_pivot.index[suggestion[i]]
            for j in books:
                if j == book_name:
                    print(f"You searched '{book_name}'\n")
                    print("The suggestion books are: \n")
                else:
                    print(j)


In [88]:
book_name = "Harry Potter and the Chamber of Secrets (Book 2)"
recommend_book(book_name)

You searched 'Harry Potter and the Chamber of Secrets (Book 2)'

The suggestion books are: 

Harry Potter and the Prisoner of Azkaban (Book 3)
Harry Potter and the Goblet of Fire (Book 4)
Harry Potter and the Sorcerer's Stone (Book 1)
Exclusive
Jacob Have I Loved
